### Reviews about Ryde

In this notebook, we are going to perform topic modelling based on sentiments from various review sites. 

In [1]:
# Base
import os, re, string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# NLTK
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.metrics.distance import edit_distance
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer

# Topic Modelling
import pyLDAvis
import pyLDAvis.gensim
import gensim
from gensim import corpora
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Data Preparation

We will first get the relevant files we need and thereafter extract them out into a list of reviews.

In [2]:
# Merge into 

path='.'

filename_sentiment_list=[]

for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Check what are the files in the folder
        print(os.path.join(path, filename))
        
        #Read the files and put to df
        df = pd.read_csv(filename,encoding = "ISO-8859-1")
        
        #Add it into a list
        try:
            filename_sentiment_list.extend(df['comment'].tolist())
            filename_sentiment_list.extend(df['tweet'].tolist())
            filename_sentiment_list.extend(df['title'].tolist())
        except:
            pass

print(filename_sentiment_list)

.\gplay_ryde_en.csv
.\reddit_rydepool_en.csv
.\reddit_rydesg_en.csv
.\reddit_ryde_en.csv
.\reddit_ryde_en_extracted.csv
.\twitter_ryde.csv
.\twitter_rydepool.csv
.\twitter_rydesg.csv
['Always enjoy te rides.Vry gd service', "don't install..full of crabs", 'Good network', 'Too many mandatory notifications that cannot be disabled. Rides more expensive or same price as Grab.', "Cannot even login when I logged out. Says my email is invalid but since my mobile number is registered, it won't allow me to login!", 'useless piece of xxxx', 'Its a good app but the problem for me is when i book i have to wait so long to get a driver please do something about it to me this app is good cause its cheaper then grab so please do something before i go back to grab', 'Not so fast to get driver. Need to catch up with Grab', 'App does not provide real-time updates on drivers location, always keep having to restart the app', 'No discount no promo', 'Nice to visit you', 'Well', 'difficult to book', 'So lous

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
filename_sentiment_list[5:7]

['useless piece of xxxx',
 'Its a good app but the problem for me is when i book i have to wait so long to get a driver please do something about it to me this app is good cause its cheaper then grab so please do something before i go back to grab']

### Cleaning

We will also perform cleaning

In [4]:
def clean_list_tokenise(reviews_list):
    # Tokenise the words
    tokenizer = RegexpTokenizer(r'\w+')
    return [tokenizer.tokenize(sentence) for sentence in reviews_list]

def clean_list_lemma(reviews_list):
    # Lemmatization
    lemma = WordNetLemmatizer()
    return [[lemma.lemmatize(word) for word in sentence] for sentence in reviews_list]

def clean_list_stopwords(reviews_list, stop_other=[]):
    def stopword_condition(word):
        word = word.lower()
        
        return word not in stopwords.words('english') \
            and word not in stop_other \
            and word[:5] != 'http' \
            and word[:5] != 'https' \
            and word[:2] != 'RT' \
            and word[0] != '@'
    
    return [[w for w in s if stopword_condition(w)] for s in reviews_list]

In [5]:
sentiment_filtered = clean_list_stopwords(clean_list_tokenise(filename_sentiment_list))
len(sentiment_filtered)

1024

In [6]:
sentiment_filtered_sentences = [' '.join(s) for s in sentiment_filtered]
sentiment_filtered_sentences[:5]

['Always enjoy te rides Vry gd service',
 'install full crabs',
 'Good network',
 'many mandatory notifications cannot disabled Rides expensive price Grab',
 'Cannot even login logged Says email invalid since mobile number registered allow login']

### Segregating by Sentiment

We will segregate the reviews by sentiment. This is achieved by the `analyse_sentiment_vader` function, which uses vader to help us analyse the degree to which the particular review or comment is positive, negative or neutral. It also computes a compound score ranging from -1 to 1 which takes into account all the scores. More reading for the sentiment analysis are available at:

- https://opensourceforu.com/2016/12/analysing-sentiments-nltk/
- http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html

In [7]:
def analyse_sentiment_vader(df, col_name):
    sid = SentimentIntensityAnalyzer()
    vader = lambda text: sid.polarity_scores(text)
    
    df['vader'] = df[col_name].apply(vader)
    df = pd.merge(df, df['vader'].apply(pd.Series), left_index=True, right_index=True)
    return df.drop(['vader'], axis=1)

In [8]:
df_sentiment_filtered = pd.DataFrame(sentiment_filtered_sentences, columns=['comment'])
df_sentiment_filtered = analyse_sentiment_vader(df_sentiment_filtered, 'comment')
df_sentiment_filtered.head(10)

,comment,neg,neu,pos,compound
0,Always enjoy te rides Vry gd service,0.000,0.652,0.348,0.4939
1,install full crabs,0.000,1.000,0.000,0.0000
2,Good network,0.000,0.256,0.744,0.4404
3,many mandatory notifications cannot disabled R...,0.000,0.860,0.140,0.0772
4,Cannot even login logged Says email invalid si...,0.000,0.775,0.225,0.2960
5,useless piece xxxx,0.583,0.417,0.000,-0.4215
6,good app problem book wait long get driver ple...,0.096,0.534,0.370,0.7717
7,fast get driver Need catch Grab,0.000,1.000,0.000,0.0000
8,App provide real time updates drivers location...,0.000,1.000,0.000,0.0000
9,discount promo,0.000,1.000,0.000,0.0000


In [9]:
df_sentiment_filtered.tail(10)

,comment,neg,neu,pos,compound
1014,Shall Thanks EDIT gave paypal call iterated FC...,0.088,0.602,0.310,0.5859
1015,said random accounts selected promo nice suppo...,0.000,0.515,0.485,0.8834
1016,category select contact,0.000,1.000,0.000,0.0000
1017,Getting error Gonna try emailing paypal hopefu...,0.201,0.597,0.201,0.0000
1018,deleted,0.000,1.000,0.000,0.0000
1019,credited 10 USD 10 SGD account spend realize t...,0.000,0.711,0.289,0.7430
1020,Sorry sure already paid perhaps chat Paypal se...,0.092,0.563,0.345,0.5574
1021,write credit 10 account,0.000,0.536,0.464,0.3818
1022,category select writing,0.000,1.000,0.000,0.0000
1023,Anyhow wack need get access form fill,0.000,1.000,0.000,0.0000


In [10]:
pos_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] >= 0]['comment'])
neg_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] < 0]['comment'])

len(pos_list), len(neg_list)

(724, 300)

In [11]:
pos_list_clean = clean_list_lemma(clean_list_tokenise(pos_list))
neg_list_clean = clean_list_lemma(clean_list_tokenise(neg_list))

In [12]:
len(pos_list_clean), len(neg_list_clean)

(724, 300)

Ignore code below

### Topic Modelling

In [36]:
neg_dict = corpora.Dictionary(neg_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in neg_list_clean]

print(doc_term_matrix[0:10])

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1)], [(3, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(12, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)], [(7, 1), (22, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)], [(7, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)], [(57, 1), (58, 1)], [(19, 1), (22, 1), (49, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)], [(19, 3), (49, 1), (54, 1), (62, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1)], [(22, 1), (66, 1), (76, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)]]


In [41]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word=neg_dict, passes=50)

In [42]:
print(ldamodel.print_topics(num_topics=2, num_words=5))

[(0, '0.017*"app" + 0.015*"driver" + 0.011*"Grab" + 0.007*"service" + 0.006*"Ryde"'), (1, '0.013*"driver" + 0.010*"grab" + 0.009*"get" + 0.009*"app" + 0.008*"Grab"')]


In [43]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, neg_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.070712  0.0       1        1  51.657761
0     -0.070712  0.0       2        1  48.342247, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
154   Default  24.000000       grab  24.000000  30.0000  30.0000
49    Default  18.000000    service  18.000000  29.0000  29.0000
62    Default  10.000000     credit  10.000000  28.0000  28.0000
31    Default  17.000000       like  17.000000  27.0000  27.0000
155   Default  10.000000   location  10.000000  26.0000  26.0000
271   Default  10.000000        got  10.000000  25.0000  25.0000
1557  Default   9.000000     market   9.000000  24.0000  24.0000
63    Default  14.000000   customer  14.000000  23.0000  23.0000
338   Default  15.000000    company  15.000000  22.0000  22.0000
1070  Default  14.000000       fake  14.000000  21.0000  21.0000
1098  Default   7.000000    phantom   7.000000  20.0000  20.0000
22    Default  62.000000        app  62.000000  19.0000  19.0000
267   Default   7.000000        day   7.000000  18.0000  18.0000
205   Default   7.000000  cancelled   7.000000  17.0000  17.0000
233   Default   9.000000     people   9.000000  16.0000  16.0000
981   Default  11.000000        com  11.000000  15.0000  15.0000
109   Default  31.000000        get  31.000000  14.0000  14.0000
616   Default   6.000000        try   6.000000  13.0000  13.0000
84    Default   7.000000    charged   7.000000  12.0000  12.0000
104   Default   6.000000        App   6.000000  11.0000  11.0000
43    Default  16.000000    booking  16.000000  10.0000  10.0000
283   Default   6.000000    Drivers   6.000000   9.0000   9.0000
72    Default  18.000000    account  18.000000   8.0000   8.0000
1071  Default   5.000000       firm   5.000000   7.0000   7.0000
165   Default   6.000000       suck   6.000000   6.0000   6.0000
26    Default   8.000000       fare   8.000000   5.0000   5.0000
232   Default   5.000000       part   5.000000   4.0000   4.0000
1143  Default   5.000000  something   5.000000   3.0000   3.0000
134   Default   6.000000        say   6.000000   2.0000   2.0000
1494  Default   4.000000         US   4.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
117    Topic2   3.707937     update   4.215474   0.5986  -6.4656
63     Topic2  12.094538   customer  14.055988   0.5766  -5.2833
981    Topic2   9.515868        com  11.247451   0.5597  -5.5231
1070   Topic2  12.353848       fake  14.764538   0.5486  -5.2621
43     Topic2  12.374836    booking  16.189913   0.4581  -5.2604
22     Topic2  39.999489        app  62.066284   0.2875  -4.0872
72     Topic2  13.598247    account  18.308140   0.4295  -5.1661
1125   Topic2   3.690876       hate   4.215757   0.5939  -6.4702
258    Topic2  13.026168       even  18.317661   0.3860  -5.2091
74     Topic2   8.120310       card  10.557810   0.4644  -5.6817
192    Topic2   6.170428    message   7.738808   0.5004  -5.9563
19     Topic2  15.124158       Ryde  22.559925   0.3270  -5.0597
334    Topic2  27.197350       Grab  48.022064   0.1583  -4.4729
7      Topic2  36.131191     driver  69.259277   0.0762  -4.1889
407    Topic2   5.100922     police   6.330880   0.5108  -6.1466
33     Topic2   9.233273      point  13.390739   0.3551  -5.5532
397    Topic2   6.715678         10   9.155459   0.4170  -5.8716
614    Topic2   6.745963     really   9.867817   0.3465  -5.8671
147    Topic2   6.417710       user   9.873281   0.2961  -5.9170
346    Topic2   5.913901    request   9.168802   0.2884  -5.9987
204    Topic2   6.814888        bad  12.005258   0.1606  -5.8569
12     Topic2  10.216915       time  29.057341  -0.3184  -5.4520
82     Topic2   7.797973        use  16.978935  -0.0512  -5.7222
109    Topic2   7.932271        get  31.946810  -0.6663  -5.7051
375    Topic2   5.236159       make   8.467216   0.2463 

# Topic Modelling (Positive)

In [31]:
pos_dict = corpora.Dictionary(pos_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in pos_list_clean]

print(doc_term_matrix[0:10])

[[(5, 1), (49, 1), (90, 1), (1897, 1)], [(691, 1)], [(1141, 1)], [(229, 1), (304, 1), (334, 1), (342, 1), (366, 1), (493, 1)], [(78, 1), (79, 1), (143, 1), (258, 1), (264, 1), (272, 1), (273, 2), (329, 1), (955, 1), (1807, 1)], [(3, 1), (7, 1), (13, 1), (22, 2), (109, 1), (154, 2), (196, 2), (354, 1), (415, 1), (432, 1), (675, 2), (1143, 2), (1559, 1), (1834, 1)], [(7, 1), (105, 1), (109, 1), (334, 1), (1516, 1)], [(7, 1), (12, 1), (22, 1), (104, 1), (114, 1), (117, 1), (129, 1), (155, 1), (162, 1), (803, 1)], [(426, 1), (483, 1)], []]


In [32]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word=pos_dict, passes=50)

In [33]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, pos_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.142757  0.004836       1        1  66.738907
1      0.065961 -0.097985       2        1  21.980207
2      0.076796  0.093149       3        1  11.280896, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
334   Default   74.000000        needed   74.000000  30.0000  30.0000
62    Default   21.000000          Nice   21.000000  29.0000  29.0000
287   Default   15.000000      compared   15.000000  28.0000  28.0000
74    Default   14.000000          load   14.000000  27.0000  27.0000
31    Default   57.000000    registered   57.000000  26.0000  26.0000
19    Default   55.000000  notification   55.000000  25.0000  25.0000
981   Default   11.000000         drive   11.000000  24.0000  24.0000
78    Default   10.000000           use   10.000000  23.0000  23.0000
338   Default   28.000000       deserve   28.000000  22.0000  22.0000
1784  Default    6.000000        heated    6.000000  21.0000  21.0000
22    Default  136.000000          Says  136.000000  20.0000  20.0000
511   Default    9.000000        select    9.000000  19.0000  19.0000
11    Default   26.000000       network   26.000000  18.0000  18.0000
1650  Default    6.000000       partner    6.000000  17.0000  17.0000
1651  Default    6.000000       upfront    6.000000  16.0000  16.0000
976   Default   12.000000          High   12.000000  15.0000  15.0000
154   Default   56.000000        search   56.000000  14.0000  14.0000
176   Default   12.000000    Definitely   12.000000  13.0000  13.0000
103   Default    6.000000          Okok    6.000000  12.0000  12.0000
524   Default    8.000000       filling    8.000000  11.0000  11.0000
601   Default    9.000000      download    9.000000  10.0000  10.0000
1447  Default   12.000000           heh   12.000000   9.0000   9.0000
1071  Default    7.000000   reliability    7.000000   8.0000   8.0000
1648  Default    5.000000        naming    5.000000   7.0000   7.0000
215   Default    9.000000      password    9.000000   6.0000   6.0000
222   Default    7.000000      starting    7.000000   5.0000   5.0000
531   Default    9.000000     GrabHitch    9.000000   4.0000   4.0000
342   Default   12.000000          take   12.000000   3.0000   3.0000
522   Default    8.000000     competing    8.000000   2.0000   2.0000
109   Default   52.000000         month   52.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
394    Topic3    1.497904            90    2.013730   1.8861  -6.1951
1183   Topic3    1.472473         Keyed    2.014955   1.8684  -6.2122
1190   Topic3    1.462566  confidential    2.038315   1.8501  -6.2190
596    Topic3    1.450812        though    2.047312   1.8377  -6.2271
1146   Topic3    2.839473          hard    4.051612   1.8266  -5.5556
1993   Topic3    1.067458         Store    1.536289   1.8180  -6.5339
497    Topic3    1.066041        ripped    1.536804   1.8163  -6.5352
1590   Topic3    1.064917   edification    1.538171   1.8144  -6.5363
380    Topic3    1.064523          well    1.538371   1.8139  -6.5367
1732   Topic3    1.063253        anyway    1.539371   1.8120  -6.5379
206    Topic3    2.431170       version    3.528556   1.8095  -5.7108
437    Topic3    1.056512       looking    1.541166   1.8045  -6.5442
673    Topic3    1.056069          note    1.542146   1.8034  -6.5446
1066   Topic3    2.729939     Uploading    4.146076   1.7642  -5.5949
166    Topic3    1.989391         quiet    3.240614   1.6941  -5.9114
640    Topic3    2.380835      greatest    4.541230   1.5363  -5.7317
601    Topic3    3.659427      download    9.154886   1.2651  -5.3019
334    Topic3   13.513093        needed   74.901329   0.4695  -3.9955
1566   Topic3    2.384418  specifically    5.329453   1.3778  -5.7302
1359   Topic3    2.155498            70    4.70